## Graph3: 🤖Bot with memory

In [6]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, START, END, add_messages
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from tavily import TavilyClient
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver

In [14]:
load_dotenv()
memory = MemorySaver()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# llm = ChatGroq(model="llama-3.1-8b-instant")

class State(TypedDict):
    messages: Annotated[list, add_messages]

# tav_key = os.environ["TAVILY_API_KEY"]


search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm_with_tools = llm.bind_tools(tools=tools)

In [3]:
def chatbot(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def router(state:State):
    last_message = state["messages"][-1]

    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return "tool_call"
    else: 
        return "end"

In [15]:
tool_node = ToolNode(tools=tools)

graph = StateGraph(State)
graph.add_node("chatbot", chatbot)
graph.add_node("tools", tool_node)

graph.set_entry_point("chatbot")
graph.add_conditional_edges("chatbot", router, {"tool_call":"tools", "end":END})
graph.add_edge("tools", "chatbot")

app = graph.compile(checkpointer=memory)
config = {"configurable":{
    "thread_id":1
}}

In [18]:
while True:
    user_input = input("👤 You: ")
    if user_input.lower() == "exit":
        print("👋 Bye Bye!")
        break
    else:
        
        result = app.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)
        
        print(f"BOT🤖: {result['messages'][-1].content}")

BOT🤖: Hello Steve! How can I help you?
BOT🤖: Yes, I am here. How can I help you, Steve?
BOT🤖: The current temperature in Vaasa, Finland is 59°F. The forecast for today includes a high of 44°F.
BOT🤖: The current temperature in Vaasa, Finland is 15°C.
BOT🤖: Today is October 27, 2024.
👋 Bye Bye!
